In [ ]:
# In this notebook I will be taking Cherie's imputed dataset (metro_CA_2011_2018.csv) and adding all of the 
# possible lags in the dependent variable from 'year_avgs.csv'

In [48]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [3]:
cherie = pd.read_csv('metro_CA_2011_2018.csv')

In [6]:
list(cherie.columns)

['Date',
 'Date.1',
 'aggregate_travel_time_to_work',
 'amerindian_including_hispanic',
 'amerindian_pop',
 'armed_forces',
 'asian_including_hispanic',
 'asian_male_45_54',
 'asian_male_55_64',
 'asian_pop',
 'associates_degree',
 'avg_rent',
 'bachelors_degree',
 'bachelors_degree_2',
 'bachelors_degree_or_higher_25_64',
 'black_including_hispanic',
 'black_male_45_54',
 'black_male_55_64',
 'black_pop',
 'children',
 'children_in_single_female_hh',
 'civilian_labor_force',
 'commute_10_14_mins',
 'commute_15_19_mins',
 'commute_20_24_mins',
 'commute_25_29_mins',
 'commute_30_34_mins',
 'commute_35_39_mins',
 'commute_35_44_mins',
 'commute_40_44_mins',
 'commute_45_59_mins',
 'commute_5_9_mins',
 'commute_60_89_mins',
 'commute_60_more_mins',
 'commute_90_more_mins',
 'commute_less_10_mins',
 'commuters_16_over',
 'commuters_by_bus',
 'commuters_by_car_truck_van',
 'commuters_by_carpool',
 'commuters_by_public_transportation',
 'commuters_by_subway_or_elevated',
 'commuters_drove_a

In [10]:
cherie['Date.1'].value_counts()

2012.0    764
2015.0    764
2014.0    764
2013.0    764
2016.0    764
2017.0    764
2011.0    764
2018.0    762
Name: Date.1, dtype: int64

In [14]:
# Aesthetic changes....
cherie = cherie.drop(columns='Date').rename(columns = {'Date.1':'year', 'Zipcode':'zipcode'})
cherie

,year,aggregate_travel_time_to_work,amerindian_including_hispanic,amerindian_pop,armed_forces,asian_including_hispanic,asian_male_45_54,asian_male_55_64,asian_pop,associates_degree,...,vacant_housing_units_for_rent,vacant_housing_units_for_sale,walked_to_work,white_including_hispanic,white_male_45_54,white_male_55_64,white_pop,worked_at_home,workers_16_and_over,zipcode
0,2011.0,797670.0,150.0,140.0,0.0,16199.0,1036.0,946.0,16104.0,2318.0,...,2017.0,278.0,7836.0,31988.0,2657.0,1716.0,28974.0,2703.0,31631.0,94109
1,2012.0,850510.0,187.0,160.0,0.0,15421.0,907.0,1002.0,15322.0,2318.0,...,1608.0,207.0,7720.0,32224.0,2614.0,1811.0,29226.0,2924.0,32836.0,94109
2,2013.0,902580.0,91.0,29.0,48.0,15639.0,880.0,1027.0,15509.0,2521.0,...,1099.0,193.0,8113.0,32779.0,2558.0,1942.0,29932.0,2829.0,33770.0,94109
3,2014.0,919860.0,85.0,16.0,74.0,15047.0,702.0,1016.0,14888.0,2353.0,...,858.0,161.0,8324.0,32432.0,2137.0,2017.0,29920.0,2585.0,34071.0,94109
4,2015.0,989955.0,96.0,13.0,55.0,15236.0,750.0,1009.0,15042.0,2336.0,...,674.0,113.0,8813.0,33489.0,2142.0,1892.0,31033.0,2648.0,35560.0,94109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6105,2013.0,2417.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,...,0.0,51.0,0.0,264.0,24.0,40.0,264.0,25.0,60.0,92341
6106,2014.0,2055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,...,0.0,59.0,0.0,294.0,23.0,38.0,294.0,23.0,85.0,92341
6107,2015.0,1950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,...,0.0,49.0,0.0,287.0,23.0,35.0,287.0,22.0,83.0,92341
6108,2016.0,1950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,...,0.0,68.0,0.0,231.0,11.0,34.0,231.0,22.0,70.0,92341


In [18]:
targets = pd.read_csv('year_avgs.csv').drop(columns='Unnamed: 0')
targets

,geo_id,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,10025,3033.75,2864.666667,2967.000000,3144.666667,3274.375000,3849.875000,4166.714286,4219.333333,3532.000000,3804.333333,NaN
1,60657,1783.75,1787.250000,1866.833333,1955.666667,2005.666667,2073.500000,2140.416667,2127.416667,2098.333333,2085.545455,1996.0
2,10023,3303.75,3074.250000,3073.833333,3198.250000,3322.250000,4153.600000,4670.375000,4032.333333,3835.125000,3960.222222,NaN
3,77494,1554.00,1584.500000,1630.583333,1786.833333,1956.833333,1921.916667,1819.833333,1770.083333,1788.166667,1763.416667,1778.0
4,60614,1919.00,1897.583333,2014.666667,2148.000000,2202.583333,2271.000000,2346.500000,2344.000000,2314.250000,2306.583333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13176,26291,946.00,957.272727,949.545455,979.272727,949.363636,1036.333333,1021.583333,949.666667,918.416667,952.250000,923.0
13177,12720,1399.00,1200.125000,1152.181818,1200.166667,1209.750000,1134.363636,1196.500000,1181.416667,1261.250000,1295.181818,1272.0
13178,26209,NaN,NaN,NaN,NaN,692.200000,769.000000,732.666667,762.666667,766.727273,739.888889,765.0
13179,41101,758.25,782.333333,846.083333,832.750000,836.166667,846.916667,865.083333,851.666667,855.916667,846.000000,815.0


In [44]:
merged = pd.merge(cherie, targets, left_on='zipcode', right_on='geo_id')

In [46]:
for i in range(merged.shape[0]):
    year = int(merged.loc[i, 'year'])
    if (year+1) in range(2010, 2021):
        merged.loc[i, 'rent_in_1y'] = merged.loc[i, str(year+1)]
    if (year+2) in range(2010, 2021):
        merged.loc[i, 'rent_in_2y'] = merged.loc[i, str(year+2)]
    if (year+3) in range(2010, 2021):
        merged.loc[i, 'rent_in_3y'] = merged.loc[i, str(year+3)]
    if (year+4) in range(2010, 2021):
        merged.loc[i, 'rent_in_4y'] = merged.loc[i, str(year+4)]
    if (year+5) in range(2010, 2021):
        merged.loc[i, 'rent_in_5y'] = merged.loc[i, str(year+5)]

In [52]:
merged.year = merged.year.astype(int)

In [53]:
merged = merged.drop(columns=['geo_id', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'])

In [54]:
merged

,year,aggregate_travel_time_to_work,amerindian_including_hispanic,amerindian_pop,armed_forces,asian_including_hispanic,asian_male_45_54,asian_male_55_64,asian_pop,associates_degree,avg_rent,bachelors_degree,bachelors_degree_2,bachelors_degree_or_higher_25_64,black_including_hispanic,black_male_45_54,black_male_55_64,black_pop,children,children_in_single_female_hh,civilian_labor_force,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,...,total_pop,two_cars,two_or_more_races_pop,two_parent_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,unemployed_pop,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,walked_to_work,white_including_hispanic,white_male_45_54,white_male_55_64,white_pop,worked_at_home,workers_16_and_over,zipcode,rent_in_1y,rent_in_2y,rent_in_3y,rent_in_4y,rent_in_5y
0,2011,797670.0,150.0,140.0,0.0,16199.0,1036.0,946.0,16104.0,2318.0,3135.416667,17652.0,16512.0,22561.0,2027.0,376.0,191.0,1679.0,3465.0,428.0,34934.0,2915.0,5160.0,4770.0,2015.0,...,53809.0,2211.0,1306.0,1346.0,543.0,730.0,28.0,45.0,2457.0,4408.0,2017.0,278.0,7836.0,31988.0,2657.0,1716.0,28974.0,2703.0,31631.0,94109,3375.500000,3747.083333,4123.250000,4520.250000,4492.916667
1,2012,850510.0,187.0,160.0,0.0,15421.0,907.0,1002.0,15322.0,2318.0,3375.500000,17652.0,17652.0,23831.0,2448.0,522.0,238.0,2132.0,3270.0,420.0,36020.0,2873.0,5125.0,5080.0,1898.0,...,54412.0,2464.0,1685.0,1256.0,508.0,671.0,34.0,43.0,2477.0,3831.0,1608.0,207.0,7720.0,32224.0,2614.0,1811.0,29226.0,2924.0,32836.0,94109,3747.083333,4123.250000,4520.250000,4492.916667,4231.416667
2,2013,902580.0,91.0,29.0,48.0,15639.0,880.0,1027.0,15509.0,2521.0,3747.083333,17866.0,17866.0,24554.0,2178.0,445.0,218.0,1865.0,3406.0,701.0,36924.0,2838.0,4920.0,5338.0,2447.0,...,55519.0,2620.0,1609.0,1358.0,653.0,625.0,38.0,42.0,2470.0,3485.0,1099.0,193.0,8113.0,32779.0,2558.0,1942.0,29932.0,2829.0,33770.0,94109,4123.250000,4520.250000,4492.916667,4231.416667,4245.916667
3,2014,919860.0,85.0,16.0,74.0,15047.0,702.0,1016.0,14888.0,2353.0,4123.250000,18818.0,18818.0,25778.0,2279.0,409.0,285.0,2221.0,2797.0,577.0,37014.0,2929.0,4938.0,5534.0,2316.0,...,54768.0,2322.0,1823.0,1116.0,425.0,625.0,38.0,28.0,2155.0,3399.0,858.0,161.0,8324.0,32432.0,2137.0,2017.0,29920.0,2585.0,34071.0,94109,4520.250000,4492.916667,4231.416667,4245.916667,4421.083333
4,2015,989955.0,96.0,13.0,55.0,15236.0,750.0,1009.0,15042.0,2336.0,4520.250000,19424.0,19424.0,27340.0,2411.0,401.0,251.0,2342.0,2914.0,669.0,38089.0,2908.0,4741.0,5912.0,2484.0,...,56293.0,2508.0,1802.0,1102.0,448.0,581.0,73.0,0.0,1673.0,3329.0,674.0,113.0,8813.0,33489.0,2142.0,1892.0,31033.0,2648.0,35560.0,94109,4492.916667,4231.416667,4245.916667,4421.083333,4406.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6105,2013,2417.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,1129.454545,81.0,81.0,77.0,0.0,0.0,0.0,0.0,61.0,25.0,94.0,0.0,0.0,0.0,0.0,...,326.0,42.0,0.0,36.0,0.0,36.0,0.0,0.0,34.0,1020.0,0.0,51.0,0.0,264.0,24.0,40.0,264.0,25.0,60.0,92341,1186.333333,1238.666667,1308.666667,1414.833333,1456.000000
6106,2014,2055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1186.333333,91.0,91.0,75.0,0.0,0.0,0.0,0.0,37.0,26.0,95.0,0.0,0.0,0.0,0.0,...,294.0,34.0,0.0,11.0,0.0,11.0,0.0,0.0,10.0,914.0,0.0,59.0,0.0,294.0,23.0,38.0,294.0,23.0,85.0,92341,1238.666667,1308.666667,1414.833333,1456.000000,1332.454545
6107,2015,1950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1238.666667,85.0,85.0,70.0,0.0,0.0,0.0,0.0,35.0,25.0,94.0,0.0,0.0,0.0,0.0,...,287.0,32.0,0.0,10.0,0.0,10.0,0.0,0.0,11.0,932.0,0.0,49.0,0.0,287.0,23.0,35.0,287.0,22.0,83.0,92341,1308.666667,1414.833333,1456.0000

In [55]:
merged.to_csv('metro_CA_2011_2018_lagged.csv', index=False)